In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.inspection import plot_partial_dependence

In [2]:
met_df = pd.read_pickle('metar_20190801_df.pkl') # Training
met_df_test = pd.read_pickle('metar_20200801_df.pkl') # Testing

In [3]:
hrrr_df = pd.read_pickle('hrrr_20190801_df.pkl')

In [ ]:
array_train = met_df_train.values
X_train = array_train[:, 1:8]
y = array_train[:, 0]

scaler = StandardScaler()
rescaled_X_train = (scaler.fit_transform(X_train[:, 3:8])) # doesn't include lon, lat

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
                   alpha=0.0001, batch_size='auto', learning_rate='adaptive', learning_rate_init=0.001,
                   power_t=0.5, max_iter=1500, shuffle=True, random_state=None, tol=0.0001, verbose=False,
                   warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                   validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp = mlp.fit(rescaled_X_train, y)

#y_predict_mlp = mlp.predict(rescaled_X_test)